In [1]:
pip install requests

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
pip install BeautifulSoup4

Defaulting to user installation because normal site-packages is not writeableNote: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.2 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import os
import lxml
from bs4 import BeautifulSoup as bs
from datetime import datetime,timedelta

In [4]:
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta
import requests
from bs4 import BeautifulSoup
import pandas as pd
import os

# 현재 날짜 구하기
now_date = datetime.now()

# 날짜정의
start_date = datetime(2018, 1, 1)
end_date = now_date
cd_min = start_date.strftime('%m/%d/%Y')
cd_max = end_date

# 함수실행
def google_news_data(query, save_path, num_pages=30):
    google_news = []
    
    for page in range(num_pages):
        # Calculate the start parameter for pagination
        start_param = page * 10
        
        # Construct the URL for the current page
        url = f"https://www.google.com/search?q={query}&tbm=nws&tbs=cdr:1,cd_min:{cd_min},cd_max:{cd_max}&start={start_param}"
        
        headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 6.1) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/84.0.4147.135 Safari/537.36'
        }

        response = requests.get(url, headers=headers)
        response.encoding = 'utf-8'
        soup = BeautifulSoup(response.text, 'html.parser')

        for con in soup.select('div.SoaBEf'):
            news_dict = {
                'links': con.find('a')['href'],
                'title': con.select_one('div.MBeuO').get_text(),
                'contents': con.select_one('.GI74Re').get_text(),
                'date': con.select_one('.LfVVr').get_text(),
                'press': con.select_one('.NUnG9d span').get_text(),
            }
            
            # 형식별로 처리
            for i in range(60):  
                
                if f'{i}시간 전' in news_dict['date']:
                    news_dict['date'] = now_date.strftime('%Y. %m. %d.')  
                
                elif f'{i}분 전' in news_dict['date']:
                    news_dict['date'] = now_date.strftime('%Y. %m. %d.')  
                                          
                elif f'{i}일 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(days=i)).strftime('%Y. %m. %d.')
                                          
                elif f'{i}주 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=i)).strftime('%Y. %m. %d.')
                                          
                elif '1개월 전' in news_dict['date']:
                    news_dict['date'] = (now_date - timedelta(weeks=4)).strftime('%Y. %m. %d.')  # 수정
            
            google_news.append(news_dict)

    df = pd.DataFrame(google_news)
    save_file_path = os.path.join(save_path, "crawling_google_search_SON.csv")
    df.to_csv(save_file_path, index=False, encoding='utf-8-sig')
    print(f"Crawling is complete. The data is saved in {save_file_path}")

    return df

query = input('Enter your search query: ')
save_path = 'C:\\DATA\\DEBA\\SON'
df = google_news_data(query, save_path, num_pages=30)  
print(df.head())


Enter your search query: 고령화
Crawling is complete. The data is saved in C:\DATA\DEBA\SON\crawling_google_search_SON.csv
                                               links  \
0  https://www.hani.co.kr/arti/economy/economy_ge...   
1        https://www.sedaily.com/NewsView/29S7E3M011   
2  https://www.shinailbo.co.kr/news/articleView.h...   
3  http://www.worktoday.co.kr/news/articleView.ht...   
4  https://news.einfomax.co.kr/news/articleView.h...   

                                      title  \
0                  '고령사회' 진입한 한국…일본보다 7년 빨라   
1  극심한 고령화에…취업자 평균연령, 47세→2050년 54세로 - 서울경제   
2         2050년 취업자 평균연령 '53.7세'…저출산·고령화 심각   
3  "일터가 빠르게 늙어간다 ... 저출산·고령화로 2050년 취업자 ...   
4    한은 "소득 불평등의 30%는 고령화 때문…사업·임대소득 격차 ...   

                                            contents           date   press  
0  저출산·고령화가 심화됨에 따라, 지난해 우리나라는 노인이 전체 인구의 14%를 웃도...   2018. 8. 27.     한겨레  
1  저출산·고령화 심화로 2050년 국내 취업자 평균 연령이 약 54세까지 높아질 것이...  2023. 08. 15.  서울경제신문  
2  저출산·고령화 현상이 심화된 탓으로 국가경